In [1]:
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import AutogluonModels
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import pickle

In [2]:
#Loaded model
target_zone='zone_18'
runtime = 600
predictor = pickle.load(open(f'Frameworks/AutoGluon/{target_zone}/{target_zone}_{runtime}g', 'rb'))

In [3]:
#Predict the missing Data between training and testing dataset
data_comlete = pd.read_csv(f'Dataset/{target_zone}.csv')
#gap is the missing values
gap = data_comlete.set_index('datetime')['2008-06-30 05:00:00': '2008-06-30 23:00:00']
gap_target = gap['target']
gap = gap.drop(columns=['target'])
pred = gap[target_zone]

In [4]:
#run loop to calculate the prediction weeks
for x in range(0, len(gap)-1):
    #row is the current hour in the loop starting with 2008-07-01 00:00:00
    row = gap[x:x+1]
    #predict the value of hour X
    pred[x]=predictor.predict(row, as_pandas=True)
    #replace the value of target_zone at hour X+1 in the feature dataset
    gap[target_zone][x+1]=pred[x]

last_value=pred[18]
print(last_value)

199695.1875


In [5]:
test = pd.read_csv(f'Dataset/Zones/{target_zone}test.csv')
#remove target from feature list
features = test.drop('target', axis=1)
#add the last prediction from the missing 19 value as the first target_column value
features[target_zone][0]=last_value
target = test['target']
pred = test[target_zone]

In [6]:
#run loop to calculate the prediction weeks
for x in range(0, len(target)):
    #row is the current hour in the loop starting with 2008-07-01 00:00:00
    row = features[x:x+1]
    #predict the value of hour X
    pred[x]=predictor.predict(row, as_pandas=True)
    #replace the value of 'zone_4' at hour X+1 in the feature dataset
    features[target_zone][x+1]=pred[x]

In [7]:
print(features[target_zone])

0      199695.187500
1      176174.875000
2      162132.125000
3      152543.921875
4      151543.671875
           ...      
162    311233.406250
163    300444.718750
164    287267.500000
165    274760.468750
166    239902.281250
Name: zone_18, Length: 167, dtype: float64


In [8]:
print(pred)

0      176174.875000
1      162132.125000
2      152543.921875
3      151543.671875
4      165450.890625
           ...      
162    300444.718750
163    287267.500000
164    274760.468750
165    239902.281250
166    204273.468750
Name: zone_18, Length: 167, dtype: float64


In [9]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import max_error

print(f'Metrics for AutoGloun {target_zone}_{runtime}')
print('Min Load:', target.min())
print('Max Load:', target.max())
print('Average Load:', target.mean())
print('R2:', r2_score(target, pred))
print('MAE:', mean_absolute_error(target, pred))
print('MAPE:', mean_absolute_percentage_error(target, pred))
print('Max Error:', max_error(target, pred))

Metrics for AutoGloun zone_18_600
Min Load: 129107.0
Max Load: 352808.0
Average Load: 225798.89221556886
R2: 0.9568953614073381
MAE: 10878.899279565869
MAPE: 0.050012556105317074
Max Error: 35966.90625
